In [110]:
import numpy as np
import tensorflow as tf
import pandas as pd
import glob

from sklearn import model_selection
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation

In [52]:
def features_from_bottles(bottles):
    X = []
    y = []
    for bottle in bottles:
        label = bottle.split("/")[-1].replace("_inception_v3.txt", "")
        line = open(bottle).readlines()
        X.append( np.loadtxt(line[0].split(','), dtype=np.float32) )
        y.append(label)
        
    X = np.array(X)
    return (X, y)
        

In [57]:
bottles = glob.glob('./features/train_img_vectors/*.txt')
X, y = features_from_bottles(bottles)

In [58]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.4)

In [87]:
train_labels = pd.read_csv('./DL3 Dataset/meta-data/train.csv')
test_labels = pd.read_csv('./DL3 Dataset/meta-data/test.csv')
train_labels = train_labels.set_index('Image_name');

In [88]:
train_labels.head(3)

,attrib_01,attrib_02,attrib_03,attrib_04,attrib_05,attrib_06,attrib_07,attrib_08,attrib_09,attrib_10,...,attrib_76,attrib_77,attrib_78,attrib_79,attrib_80,attrib_81,attrib_82,attrib_83,attrib_84,attrib_85
Image_name,,,,,,,,,,,,,,,,,,,,,
Image-1.jpg,0,0,0,0,0,1,1,0,0,0,...,1,0,0,0,0,0,0,0,0,1
Image-2.jpg,1,0,0,1,0,0,1,1,0,0,...,1,1,1,1,0,0,0,1,0,1
Image-3.jpg,1,0,0,1,0,0,0,1,1,0,...,1,0,0,1,0,1,0,1,0,1


In [94]:
len(train_labels.loc['Image-22.jpg'].tolist())

85

In [101]:
y_train_ = np.array([train_labels.loc[label] for label in y_train])
y_test_ = np.array([train_labels.loc[label] for label in y_test])

In [124]:
model = Sequential()
model.add(Dense(2048, activation='tanh', input_dim=X_train.shape[1]))
model.add(Dropout(0.5))
model.add(Dense(1024, activation='tanh'))
model.add(Dropout(0.5))
model.add(Dense(y_train_.shape[1], activation='sigmoid'))
model.compile(loss='binary_crossentropy',
              optimizer='adam')

model.fit(X_train, y_train_, epochs=15, batch_size=2000)

Epoch 1/15
7560/7560 [==============================] - 3s 447us/step - loss: 0.6221
Epoch 2/15
7560/7560 [==============================] - 3s 380us/step - loss: 0.4611
Epoch 3/15
7560/7560 [==============================] - 3s 382us/step - loss: 0.3340
Epoch 4/15
7560/7560 [==============================] - 3s 408us/step - loss: 0.2594
Epoch 5/15
7560/7560 [==============================] - 3s 407us/step - loss: 0.2170
Epoch 6/15
7560/7560 [==============================] - 3s 397us/step - loss: 0.1877
Epoch 7/15
7560/7560 [==============================] - 3s 399us/step - loss: 0.1646
Epoch 8/15
7560/7560 [==============================] - 3s 402us/step - loss: 0.1466
Epoch 9/15
7560/7560 [==============================] - 3s 399us/step - loss: 0.1333
Epoch 10/15
7560/7560 [==============================] - 3s 407us/step - loss: 0.1215
Epoch 11/15
7560/7560 [==============================] - 3s 423us/step - loss: 0.1127
Epoch 12/15
7560/7560 [==============================] - 3s 407

In [125]:
preds = model.predict(X_test)
preds[preds>=0.5] = 1
preds[preds<0.5] = 0

result = preds.reshape(-1) == y_test_.reshape(-1)
acc_on_test = sum(result)/len(result)
print("Accuracy", acc_on_test)

Accuracy 0.976360877684407


0.9712581699346405

In [126]:
round(0.976360877684407, 3)

0.976

### Test images from Hacker Earth

In [127]:
bottles = glob.glob('./features/test_img_vectors/*.txt')
X, y = features_from_bottles(bottles)

In [129]:
preds = model.predict(X)
preds[preds>=0.5] = 1
preds[preds<0.5] = 0

In [133]:
preds.shape

(5400, 85)

In [147]:
attribs = ['attrib_'+"{:02}".format(i) for i in range(1, 86)]

In [176]:
data = {}
for i in range(len(preds)):
    row = {a: int(b)  for (a,b) in zip(attribs, preds[i])}
    data[y[i]] = row

In [178]:
result = []
for val in test_labels['Image_name'].tolist():
    point_ = data[val]
    point_['Image_name'] = val
    result.append(point_)
    

In [181]:
df_preds = pd.DataFrame(result)
df_preds = df_preds.set_index('Image_name')

In [182]:
df_preds.head()

,attrib_01,attrib_02,attrib_03,attrib_04,attrib_05,attrib_06,attrib_07,attrib_08,attrib_09,attrib_10,...,attrib_76,attrib_77,attrib_78,attrib_79,attrib_80,attrib_81,attrib_82,attrib_83,attrib_84,attrib_85
Image_name,,,,,,,,,,,,,,,,,,,,,
Image-1.jpg,0,0,0,0,0,1,0,0,1,1,...,0,0,0,1,1,0,0,1,0,1
Image-2.jpg,0,1,0,1,0,1,1,0,0,0,...,0,1,0,1,0,0,0,0,1,1
Image-3.jpg,0,0,0,1,0,0,1,0,0,0,...,1,0,0,1,0,0,0,1,0,1
Image-4.jpg,1,0,0,0,1,1,1,0,0,1,...,0,1,1,1,0,0,0,0,0,0
Image-5.jpg,0,0,0,1,0,1,0,0,0,1,...,0,0,0,1,1,0,0,1,0,0


In [183]:
df_preds.to_csv('result.csv')